# Multi Layer Perceptron (MLP)

> Simple feedforward Multilayer perceptron model

In [24]:
#| default_exp models.mlp

In [25]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
#| export
import torch.nn as nn
import torch
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST


from pytorch_lightning import LightningModule, Trainer
from torchmetrics import Accuracy
from hydra.utils import instantiate
from omegaconf import OmegaConf

from nimrod.data.datasets import MNISTDataModule
from nimrod.utils import get_device

from IPython.core.debugger import set_trace

## Basic model

In [27]:
#| export
class MLP(nn.Module):
    def __init__(
                self, n_in:int=32*32*3, # input dimension e.g. (H,W) for image
                n_h:int=64, # hidden dimension
                n_out:int=10 # output dimension (= number of classes for classification)
                ):
        super().__init__()
        l1 = nn.Linear(n_in, n_h)
        l2 = nn.Linear(n_h, n_out)
        dropout = nn.Dropout(0.2)
        self.layers = nn.Sequential(l1,l2, dropout)
        
    def forward(self, x: torch.FloatTensor # dim (B, H*W)
                ) -> torch.FloatTensor:
        return self.layers(x)

### Usage

In [28]:
image = torch.rand((5, 28*28))
mlp = MLP(n_in=28*28, n_h=64, n_out=10)
out = mlp(image)
print(out.shape)

torch.Size([5, 10])


### Basic training
#### Data Module
Data module
c.f. recipes/image/mnist

In [29]:
!cat ../config/data/image/mnist.yaml

69.07s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
dataset:
  _target_: nimrod.image.datasets.MNISTDataset
  data_dir: "../data/image"
  train: False
  transform: 
    _target_: torchvision.transforms.ToTensor

datamodule:
  _target_: nimrod.image.datasets.MNISTDataModule
  data_dir: "../data/image"
  train_val_test_split: [0.8, 0.1, 0.1]
  batch_size: 64
  num_workers: 0
  pin_memory: False


In [30]:
# load from config file
cfg = OmegaConf.load('../config/data/image/mnist.yaml')
datamodule = instantiate(cfg.datamodule)
datamodule.prepare_data()
datamodule.setup()
x = datamodule.data_test[0][0] # (C, H, W)
print(len(datamodule.data_test))
label = datamodule.data_test[0][1] #(int)
print("original shape (C,H,W): ", x.shape)
print("reshape (C,HxW): ", x.view(x.size(0), -1).shape)
print(x[0][1])

7000
original shape (C,H,W):  torch.Size([1, 28, 28])
reshape (C,HxW):  torch.Size([1, 784])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])


In [31]:
# using default Pytorch datasets
train_dataset = MNIST("../data/image", train=True, download=True, transform=ToTensor())
test_dataset = MNIST("../data/image", train=False, download=True, transform=ToTensor())
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# using nimrod datamodule
train_loader = datamodule.train_dataloader()
val_loader = datamodule.val_dataloader()
test_loader = datamodule.test_dataloader()

#### Hardware acceleration

In [32]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
device = torch.device(device)
model = mlp.to(device)

#### Loss & optimizer setup

In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

#### Training loop

In [34]:
%%time
n_epochs = 1
for epoch in range(n_epochs):
    model.train()
    for images, labels in train_loader:
        images = images.view(-1, 28*28)
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        # print(loss.item())
        optimizer.step()

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            # model expects input (B,H*W)
            images = images.view(-1, 28*28).to(device)
            images = images.to(device)
            labels = labels.to(device)
            # Pass the input through the model
            outputs = model(images)
            # Get the predicted labels
            _, predicted = torch.max(outputs.data, 1)

            # Update the total and correct counts
            total += labels.size(0)
            correct += (predicted == labels).sum()

        # Print the accuracy
        print(f"Epoch {epoch + 1}: Accuracy = {100 * correct / total:.2f}%")

Epoch 1: Accuracy = 74.00%
CPU times: user 7.86 s, sys: 492 ms, total: 8.35 s
Wall time: 8.57 s


## Integrated model + training settings

In [35]:
#| export
class MLP_PL(LightningModule):
    def __init__(self, mlp:MLP):
        super().__init__()
        self.save_hyperparameters(ignore=['mlp'])
        self.mlp = mlp
        self.loss = nn.CrossEntropyLoss()
        self.accuracy = Accuracy(task="multiclass", num_classes=10)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def forward(self, x):
        return(self.mlp(x))

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = x.view(x.size(0), -1)
        y_hat = self.mlp(x)
        print("Y:", y_hat, y)
        loss = self.loss(y_hat, torch.LongTensor([y]))
        return loss
    
    def _step(self, batch, batch_idx):
        x, y = batch
        x = x.view(x.size(0), -1)
        y_hat = self.mlp(x)
        loss = self.loss(y_hat, y)
        acc = self.accuracy(y_hat, y)
        return loss, acc
    
    def validation_step(self, batch, batch_idx, prog_bar=True, on_step=False, on_epoch=True, sync_dist=True):
        loss, acc = self._step(batch, batch_idx)
        metrics = {"val/loss":loss, "val/acc": acc}
        self.log_dict(metrics, on_step=on_step, on_epoch=on_epoch, sync_dist=sync_dist)
    
    def test_step(self, batch, batch_idx, prog_bar=True, on_step=False, on_epoch=True, sync_dist=True):
        loss, acc = self._step(batch, batch_idx)
        metrics = {"test/loss":loss, "test/acc": acc}
        self.log_dict(metrics, on_step=on_step, on_epoch=on_epoch, sync_dist=sync_dist)

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch
        y_hat = self.mlp(x)
        return y_hat


### Usage

In [36]:
# wrap simple model in modularized model
mlp_pl = MLP_PL(mlp)
# fake input
b = torch.rand((5,28*28))

# move model and data to hardware
model = mlp_pl.to(device)
b = b.to(device)

y = mlp_pl(b)
print(y.shape)

torch.Size([5, 10])


## Integrated trainer

In [ ]:
trainer = Trainer(accelerator='cpu', fast_dev_run=True) #mps', devices=1)
trainer.fit(mlp_pl, datamodule=datamodule)
# trainer.fit(mlp_pl, datamodule.data_train)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()